In [21]:
#İsmail Burak Kurhan
#220201055
import numpy as np
import cv2
import sys
import math
from numpy.linalg import inv

img = cv2.imread("img1.png",0)
t =50   # Scale parameter,
u =30   # In-plane rotation angle
l =0.5  # Tilt amount

sint= math.sin(math.radians(t))
cost = math.cos(math.radians(t))
sinu = math.sin(math.radians(u))
cosu = math.cos(math.radians(u))

matrixAffine = l*(np.dot(np.array([[cosu,sinu],[-sinu,cosu]]),np.array([[cost, 0], [0, 1]])))

cornerbr = np.dot(matrixAffine,np.array([[len(img)],[len(img[0])]]))
cornerbl = np.dot(matrixAffine,np.array([[0],[len(img[0])]]))
cornertr = np.dot(matrixAffine,np.array([[len(img)],[0]]))
cornertl = np.dot(matrixAffine,np.array([[0],[0]]))

height = abs(int((cornertr[1])-(cornerbl[1])))
width = abs(int((cornerbr[0])-(cornertl[0])))

Warped_Image = np.zeros([width, height,1])  #Create Frame of the warped image

matrixHomogrophy1 = np.dot(np.matrix([[1,0,width/2],[0,1,height/2],[0,0,1]]),np.matrix([[matrixAffine[0][0],matrixAffine[0][1],0],[matrixAffine[1][0],matrixAffine[1][1],0],[0,0,1]]))
matrixHomogrophy2 = np.dot(matrixHomogrophy1,np.matrix([[1,0,-len(img)/2],[0,1,-len(img[0])/2],[0,0,1]]))

homogrophyInverse = inv(matrixHomogrophy2)

referencelist = []
i = 0
while i < width:    
    j = 0
    while j < height:
        referencelist.append((np.dot(homogrophyInverse,np.array([[i],[j],[1]]))))
        #---------find nearest referenced point-----------
        firstpointx =  round(float(referencelist[0][0]))
        firstpointy =  round(float(referencelist[0][1]))
        #------------Check out of bound------------------------
        if ( firstpointx>=len(img)-1):
            Warped_Image[i][j]=0
            j+=1
            del referencelist[0]
            continue
        elif (firstpointy>=len(img[0])-1):
            Warped_Image[i][j]=0
            j+=1
            del referencelist[0]
            continue
        elif (firstpointx < 0 ):
            Warped_Image[i][j]=0
            j+=1
            del referencelist[0]
            continue
        elif (firstpointy < 0):
            Warped_Image[i][j]=0
            j+=1
            del referencelist[0]
            continue
  
        x_ratio= (referencelist[0][0] - firstpointx)
        y_ratio= (referencelist[0][1] - firstpointy)
        #------------------bilinear-interpolation-------------------------------
        a=  (1 - x_ratio) * img[firstpointx][firstpointy] * (1-y_ratio)
        b=  img[firstpointx+1][firstpointy]* x_ratio* (1-y_ratio)
        c= y_ratio *img[firstpointx][firstpointy+1]*  (1 - x_ratio)
        d= y_ratio * x_ratio * img[firstpointx+1][firstpointy+1]
       
        intensity = a + b + c +d

        Warped_Image[i][j]=intensity
        del referencelist[0]
        j+=1
    i+=1

cv2.imwrite("warped_img.png", Warped_Image)

True

In [ ]:
#COMMENT-1 :I'm using inverse homogrophy for takes intesities of warped image from reference image.
# By this multiplaction with left side and right side we can detect true coordinates of images.
# because we are giving width and height of reference image with left side matrix and width and height of warped image
# with right side matrix.
# If the don't multiply these matrices with homogrophy matrix our warped image will not fit the frame.
# so, some part of the warped images take place at the out of frame.
# due to this reason we are multiplying affine matrix with two matrices

In [ ]:
#COMMENT-2 : We transform warped image to the reference image with inverse homogrophy because our aim is find the
# correct coordinates of reference image which refer to warped image. This code starts with calculating frame of the
# warped image, after finding corresponding coordinates with inverse homogrophy. If we calculate corresponding coordinates
# with homogrophy matrix from reference image to warped image maybe we will can not find some intensity of pixels for different affine transformations.
# By this method we are finding the correct intensity values of warped image.

In [ ]:
# COMMENT-3 : bilinear interpolation compute intensity values more close to the right intensity.
# because bilinear interpolation check nearest 4 pixel and calculate intensity with their distance rate.
# If we use nearest-neighbour interpolation instead of bilinear interpolation gives more bad result because 
# nearest-neighbour interpolation check only one pixel and takes this intensity instead of 4 pixel.